In [14]:
import time

# **Multiprocessing**

In [16]:
from multiprocessing import Pool
import math

def f(x):
    return math.sqrt(1 - x**2)

def partial_sum(args):
    start, end, dx = args
    subtotal = 0
    for i in range(start, end):
        xi = i * dx
        subtotal += f(xi) * dx
    return subtotal

def compute_pi(N, num_processes):
    dx = 1.0 / N
    ranges = [(int(i * N / num_processes), int((i + 1) * N / num_processes), dx) for i in range(num_processes)]
    with Pool(processes=num_processes) as pool:
        results = pool.map(partial_sum, ranges)
    return 4 * sum(results)

In [17]:
# Ejemplo de uso
N = 1000000
num_processes = 4

start_time = time.time()
pi_approx = compute_pi(N, num_processes)
end_time = time.time()

print(f"Approximated pi = {pi_approx}")
print(f"Execution Time: {end_time - start_time} seconds")

Approximated pi = 3.141594652413769
Execution Time: 0.4386017322540283 seconds


# **Paralelization**

In [18]:
import math

def f(x):
    return math.sqrt(1 - x**2)

def compute_pi(N):
    total_sum = 0
    dx = 1.0 / N
    for i in range(N):
        xi = i * dx
        total_sum += f(xi) * dx
    return 4 * total_sum

In [19]:
N = 1000000

start_time = time.time()
pi_approx = compute_pi(N)
end_time = time.time()

print(f"Approximated pi = {pi_approx}")
print(f"Execution Time: {end_time - start_time} seconds")

Approximated pi = 3.1415946524138207
Execution Time: 0.3948206901550293 seconds


# **Mpi4py**

In [6]:
!pip install mpi4py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 30.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.6-cp310-cp310-linux_x86_64.whl size=2746302 sha256=d5c18e16a1637cb450631255865e6cbeaf8a48b357d1999f14b3cb18843a5d15
  Stored in directory: /root/.cache/pip/wheels/4c/ca/89/8fc1fb1c620afca13bb41c630b1f948bbf446e0aaa4b762e10
Successfully built mpi4py


In [23]:
from mpi4py import MPI
import math

def f(x):
    return math.sqrt(1 - x**2)

def compute_pi_mpi(N):
    comm = MPI.COMM_WORLD
    size = comm.Get_size()
    rank = comm.Get_rank()

    dx = 1.0 / N
    local_n = N // size
    local_start = rank * local_n
    local_end = local_start + local_n if rank != size - 1 else N

    local_start_time = MPI.Wtime()

    local_sum = sum(f(i * dx) * dx for i in range(local_start, local_end))
    total_sum = comm.reduce(local_sum, op=MPI.SUM, root=0)

    local_end_time = MPI.Wtime()

    max_time = comm.reduce(local_end_time - local_start_time, op=MPI.MAX, root=0)

    if rank == 0:
        return 4 * total_sum, max_time

In [24]:
if __name__ == "__main__":
    N = 1000000
    result = compute_pi_mpi(N)
    if MPI.COMM_WORLD.Get_rank() == 0:
        pi_approx, max_execution_time = result
        print(f"Approximated pi = {pi_approx}")
        print(f"Execution Time: {max_execution_time} seconds")

Approximated pi = 3.1415946524138207
Execution Time: 0.3725428569999849 seconds
